In [40]:
import polars as pl
import openai
import unicodedata
import re
import os
from dotenv import load_dotenv
load_dotenv()

data = pl.read_csv("./data.csv", separator=";")
data

category_coarse,category_fine,query_text,priority,tags,answer_pattern
str,str,str,str,str,str
"""Новые клиенты""","""Регистрация и онбординг""","""Как стать клиентом банка онлай…","""высокий""","""новые клиенты""","""Стать клиентом ВТБ (Беларусь) …"
"""Новые клиенты""","""Регистрация и онбординг""","""Регистрация через МСИ (Межбанк…","""высокий""","""новые клиенты""","""МСИ позволяет пройти идентифик…"
"""Новые клиенты""","""Регистрация и онбординг""","""Документы для регистрации ново…","""высокий""","""новые клиенты""","""Для регистрации в качестве нов…"
"""Новые клиенты""","""Первые шаги""","""Первый вход в Интернет-банк""","""высокий""","""новые клиенты""","""После регистрации вы получите …"
"""Новые клиенты""","""Первые шаги""","""Как скачать и настроить мобиль…","""высокий""","""новые клиенты""","""Мобильное приложение VTB mBank…"
…,…,…,…,…,…
"""Частные клиенты""","""Онлайн-сервисы""","""Система пишет, что такого клие…","""средний""","""все клиенты""","""Перед началом использования си…"
"""Частные клиенты""","""Онлайн-сервисы""","""А логин и пароль в новом Интер…","""средний""","""все клиенты""","""Нет, Клиент может, используя о…"
"""Частные клиенты""","""Онлайн-сервисы""","""Есть ли возможность принимать …","""средний""","""все клиенты""","""Платежи сервиса E-POS доступны…"


In [10]:
def normalize_text(s: str) -> str:
    # lowercasing, unicode normalization, removal of weird punctuation, extra whitespace
    if s is None:
        return ""
    # normalize unicode (e.g. cyrillic normalization)
    s = unicodedata.normalize("NFKC", s)
    s = s.lower()
    # remove extra spaces
    s = re.sub(r"\s+", " ", s).strip()
    # optional: remove non-cyrillic/latin punctuation or map them
    return s

def tokenize_simple(s: str) -> list[str]:
    # simple whitespace + punctuation split
    # you can expand with better tokenization (morphology, ru / by lexicons, etc.)
    s2 = re.sub(r"[^\wа-яёА-ЯЁ]+", " ", s)
    return s2.split()

In [41]:

client = openai.OpenAI(api_key=os.getenv("SCIBOX_API_KEY"), base_url="https://llm.t1v.scibox.tech/v1")


In [14]:
df = data.with_columns([
    pl.col("query_text").map_elements(normalize_text).alias("query_norm"),
    pl.col("answer_pattern").map_elements(normalize_text).alias("answer_norm"),
    pl.col("category_coarse").map_elements(normalize_text).alias("cat_coarse_norm"),
    pl.col("category_fine").map_elements(normalize_text).alias("cat_fine_norm"),
])
df

category_coarse,category_fine,query_text,priority,tags,answer_pattern,query_norm,answer_norm,cat_coarse_norm,cat_fine_norm
str,str,str,str,str,str,str,str,str,str
"""Новые клиенты""","""Регистрация и онбординг""","""Как стать клиентом банка онлай…","""высокий""","""новые клиенты""","""Стать клиентом ВТБ (Беларусь) …","""как стать клиентом банка онлай…","""стать клиентом втб (беларусь) …","""новые клиенты""","""регистрация и онбординг"""
"""Новые клиенты""","""Регистрация и онбординг""","""Регистрация через МСИ (Межбанк…","""высокий""","""новые клиенты""","""МСИ позволяет пройти идентифик…","""регистрация через мси (межбанк…","""мси позволяет пройти идентифик…","""новые клиенты""","""регистрация и онбординг"""
"""Новые клиенты""","""Регистрация и онбординг""","""Документы для регистрации ново…","""высокий""","""новые клиенты""","""Для регистрации в качестве нов…","""документы для регистрации ново…","""для регистрации в качестве нов…","""новые клиенты""","""регистрация и онбординг"""
"""Новые клиенты""","""Первые шаги""","""Первый вход в Интернет-банк""","""высокий""","""новые клиенты""","""После регистрации вы получите …","""первый вход в интернет-банк""","""после регистрации вы получите …","""новые клиенты""","""первые шаги"""
"""Новые клиенты""","""Первые шаги""","""Как скачать и настроить мобиль…","""высокий""","""новые клиенты""","""Мобильное приложение VTB mBank…","""как скачать и настроить мобиль…","""мобильное приложение vtb mbank…","""новые клиенты""","""первые шаги"""
…,…,…,…,…,…,…,…,…,…
"""Частные клиенты""","""Онлайн-сервисы""","""Система пишет, что такого клие…","""средний""","""все клиенты""","""Перед началом использования си…","""система пишет, что такого клие…","""перед началом использования си…","""частные клиенты""","""онлайн-сервисы"""
"""Частные клиенты""","""Онлайн-сервисы""","""А логин и пароль в новом Интер…","""средний""","""все клиенты""","""Нет, Клиент может, используя о…","""а логин и пароль в новом интер…","""нет, клиент может, используя о…","""частные клиенты""","""онлайн-сервисы"""
"""Частные клиенты""","""Онлайн-сервисы""","""Есть ли возможность принимать …","""средний""","""все клиенты""","""Платежи сервиса E-POS доступны…","""есть ли возможность принимать …","""платежи сервиса e-pos доступны…","""частные клиенты""","""онлайн-сервисы"""


In [ ]:
# A batched version (for speed)
def get_embeddings_batch(texts: list[str], model: str = "bge-m3") -> list[list[float]]:
    resp = client.embeddings.create(
        model=model,
        input=texts
    )
    embs = [d.embedding for d in resp.data]
    return embs

queries = df.select("query_norm").unique().to_series().to_list()
embs = get_embeddings_batch(queries)
# Create a lookup map
query_to_emb = {q: e for q, e in zip(queries, embs)}
print(type(query_to_emb))


<class 'dict'>


AttributeError: 'str' object has no attribute 'to_list'

In [32]:

def query_to_emb_as_list(q: str) -> list[float]:
    return query_to_emb.get(q, None)

df = df.with_columns([
    pl.col("query_norm").map_elements(query_to_emb_as_list, return_dtype=list[float]).alias("query_emb")
])
df

category_coarse,category_fine,query_text,priority,tags,answer_pattern,query_norm,answer_norm,cat_coarse_norm,cat_fine_norm,query_emb
str,str,str,str,str,str,str,str,str,str,list[f64]
"""Новые клиенты""","""Регистрация и онбординг""","""Как стать клиентом банка онлай…","""высокий""","""новые клиенты""","""Стать клиентом ВТБ (Беларусь) …","""как стать клиентом банка онлай…","""стать клиентом втб (беларусь) …","""новые клиенты""","""регистрация и онбординг""","[-0.044454, 0.004914, … -0.051136]"
"""Новые клиенты""","""Регистрация и онбординг""","""Регистрация через МСИ (Межбанк…","""высокий""","""новые клиенты""","""МСИ позволяет пройти идентифик…","""регистрация через мси (межбанк…","""мси позволяет пройти идентифик…","""новые клиенты""","""регистрация и онбординг""","[-0.003638, -0.016539, … -0.008411]"
"""Новые клиенты""","""Регистрация и онбординг""","""Документы для регистрации ново…","""высокий""","""новые клиенты""","""Для регистрации в качестве нов…","""документы для регистрации ново…","""для регистрации в качестве нов…","""новые клиенты""","""регистрация и онбординг""","[-0.041469, -0.01118, … -0.006432]"
"""Новые клиенты""","""Первые шаги""","""Первый вход в Интернет-банк""","""высокий""","""новые клиенты""","""После регистрации вы получите …","""первый вход в интернет-банк""","""после регистрации вы получите …","""новые клиенты""","""первые шаги""","[0.014297, 0.009752, … -0.046199]"
"""Новые клиенты""","""Первые шаги""","""Как скачать и настроить мобиль…","""высокий""","""новые клиенты""","""Мобильное приложение VTB mBank…","""как скачать и настроить мобиль…","""мобильное приложение vtb mbank…","""новые клиенты""","""первые шаги""","[-0.023836, -0.011999, … -0.040541]"
…,…,…,…,…,…,…,…,…,…,…
"""Частные клиенты""","""Онлайн-сервисы""","""Система пишет, что такого клие…","""средний""","""все клиенты""","""Перед началом использования си…","""система пишет, что такого клие…","""перед началом использования си…","""частные клиенты""","""онлайн-сервисы""","[-0.072425, -0.00813, … -0.008906]"
"""Частные клиенты""","""Онлайн-сервисы""","""А логин и пароль в новом Интер…","""средний""","""все клиенты""","""Нет, Клиент может, используя о…","""а логин и пароль в новом интер…","""нет, клиент может, используя о…","""частные клиенты""","""онлайн-сервисы""","[0.001372, -0.020665, … -0.047109]"
"""Частные клиенты""","""Онлайн-сервисы""","""Есть ли возможность принимать …","""средний""","""все клиенты""","""Платежи сервиса E-POS доступны…","""есть ли возможность принимать …","""платежи сервиса e-pos доступны…","""частные клиенты""","""онлайн-сервисы""","[-0.02139, 0.013127, … -0.049746]"


In [ ]:
from sklearn.metrics import accuracy_score, classification_report, f1_score, precision_recall_fscore_support, top_k_accuracy_score

from sklearn.model_selection import train_test_split
from sklearn.neighbors import NearestNeighbors
import numpy as np
y = df["category_fine"].to_list()
X = df["query_emb"].to_list()
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# --- 3) OpenAI embeddings (batched) -----------------------------------------


# L2-normalize for cosine via dot product trick
def l2norm(a):
    n = np.linalg.norm(a, axis=1, keepdims=True) + 1e-9
    return a / n

E_train = l2norm(X_train)
E_test  = l2norm(X_test)


Accuracy:  0.5366
Macro F1:  0.3870

Per-class report:
                                           precision    recall  f1-score   support

     Автокредиты - Автокредит без залога     0.0000    0.0000    0.0000         1
                     Банковские карточки     0.8000    1.0000    0.8889         4
                          Валютные - CNY     0.0000    0.0000    0.0000         0
                          Валютные - EUR     0.0000    0.0000    0.0000         1
                          Валютные - RUB     0.0000    0.0000    0.0000         1
                          Валютные - USD     0.0000    0.0000    0.0000         1
                       Вклады и депозиты     0.5000    0.3333    0.4000         3
              Дебетовые карты - Infinite     1.0000    1.0000    1.0000         1
                  Дебетовые карты - MORE     1.0000    1.0000    1.0000         2
             Дебетовые карты - Signature     1.0000    1.0000    1.0000         1
            Дебетовые карты - Комплимент 

/Users/paul/Projects/t1-smart-support/.venv/lib/python3.13/site-packages/sklearn/metrics/_classification.py:98: UserWarning: The number of unique classes is greater than 50% of the number of samples. `y` could represent a regression problem, not a classification problem.
  type_true = type_of_target(y_true, input_name="y_true")
/Users/paul/Projects/t1-smart-support/.venv/lib/python3.13/site-packages/sklearn/metrics/_classification.py:99: UserWarning: The number of unique classes is greater than 50% of the number of samples. `y` could represent a regression problem, not a classification problem.
  type_pred = type_of_target(y_pred, input_name="y_pred")
/Users/paul/Projects/t1-smart-support/.venv/lib/python3.13/site-packages/sklearn/metrics/_classification.py:98: UserWarning: The number of unique classes is greater than 50% of the number of samples. `y` could represent a regression problem, not a classification problem.
  type_true = type_of_target(y_true, input_name="y_true")
/Users/pau

In [34]:
print(y_train)

['Потребительские - На всё про всё', 'Вклады и депозиты', 'Вклады и депозиты', 'Дебетовые карты - Форсаж', 'Карты рассрочки - ЧЕРЕПАХА', 'Кредиты', 'Валютные - USD', 'Банковские карточки', 'Дебетовые карты - Форсаж', 'Проблемы и решения', 'Вклады и депозиты', 'Банковские карточки', 'Экспресс-кредиты - В магазинах-партнерах', 'Дебетовые карты - MORE', 'Кредитные карты - PLAT/ON', 'Банковские карточки', 'Дебетовые карты - MORE', 'Экспресс-кредиты - На роднае', 'Банковские карточки', 'Рублевые - Подушка безопасности', 'Банковские карточки', 'Потребительские - Легко платить', 'Дебетовые карты - MORE', 'Проблемы и решения', 'Валютные - EUR', 'Потребительские - Всё только начинается', 'Кредиты', 'Дебетовые карты - Infinite', 'Рублевые - Мои условия', 'Рублевые - СуперСемь', 'Карты рассрочки - КСТАТИ', 'Кредитные карты - PLAT/ON', 'Рублевые - Великий путь', 'Кредиты', 'Валютные - CNY', 'Банковские карточки', 'Кредитные карты - Портмоне 2.0', 'Банковские карточки', 'Дебетовые карты - Форсаж', 

In [ ]:

# --- 4) Simple kNN (cosine) to get y_pred (top-1) ---------------------------
# For cosine with NearestNeighbors, use metric="cosine" (it returns distances = 1 - cosine_sim)
knn = NearestNeighbors(n_neighbors=1, metric="cosine")
knn.fit(E_train)

dist, idx = knn.kneighbors(E_test, return_distance=True)   # idx: (n_test, 1)
# Map neighbor index to label
y_train_arr = np.array(y_train)
y_pred = y_train_arr[idx[:,0]].tolist()

# --- 5) Metrics: y_true / y_pred -------------------------------------------
y_true = y_test  # for clarity

acc = accuracy_score(y_true, y_pred)
macro_f1 = f1_score(y_true, y_pred, average="macro")

print(f"Accuracy:  {acc:.4f}")
print(f"Macro F1:  {macro_f1:.4f}")
print("\nPer-class report:\n", classification_report(y_true, y_pred, digits=4))